## Experiments tested :

[1] Fuzzy sphere

[2] Higher dimensional synthetic manifolds
- M10b
- MBeta
- MN1

In [1]:
import skdim
import plotly.express as px
import torch
from torch.utils.data import DataLoader
from qcml.model import QuantumCognitionModel, train
from qcml.intrinsic_dimension import intrinsic_dimension
from qcml.utils import plot_sphere
seed = 42
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
bm = skdim.datasets.BenchmarkManifolds(random_state=seed)

## 1. Fuzzy sphere

Collecting data and training QCML model

In [ ]:
D = 3
d = 2
N = 3
n = 2500
X = bm.generate(name='M1_Sphere', dim=D, d=d, noise=0.1, n=n)
model = QuantumCognitionModel(N, N, 0.).to(device)
model = train(model, X, {"lr": 1e-3, "n_epochs": 50})
X_A = model(torch.tensor(X).to(device))[0].real

Plot of raw noisy data and manifold found.

In [ ]:
plot_sphere(X, X_A.cpu().detach().numpy())

Below a screenshot of the fitting observed.

![alt text](img/hypersphere.png "Title")

We conclude by calculating the intrinsic dimension found for the manifold through searching for biggest gaps in the eigenvalues of the quantum metric.

In [4]:
dlist = intrinsic_dimension(model, X_A)
print(f"Average dimension found over dataset : {torch.mean(dlist, dtype=float).item()} (True: {d})")

Average dimension found over dataset : 2.0 (True: 2)


## 2. Higher dimensional synthetic manifolds

For each of the samples below we perform the same steps as above : 
- collecting data
- training QCML model
- calculating intrinsic dimension of manifold found

### M10b

17 dimensions linear manifold inside space of dimension 18.

In [ ]:
D = 18
d = 17
N = 16
n = 2500
X = bm.generate(name='M10b_Cubic', dim=D, d=d, noise=0.1, n=n)
model = QuantumCognitionModel(D, N, 0.).to(device)
model = train(model, X, {"lr": 1e-3, "n_epochs": 50})
X_A = model(torch.tensor(X).to(device))[0].real

In [6]:
dlist = torch.cat([intrinsic_dimension(model, x) for x in DataLoader(X_A, batch_size=256)])
print(f"Average dimension found over dataset : {torch.mean(dlist, dtype=float).item()} (True: {d})")

Average dimension found over dataset : 16.9428 (True: 17)


### Mbeta

10 dimensions non-linear manifold inside space of dimension 40.

In [ ]:
D = 40
d = 10
N = 16
n = 250
X = bm.generate(name='Mbeta', dim=D, d=d, noise=0.1, n=n)
model = QuantumCognitionModel(D, N, 0.).to(device)
model = train(model, X, {"lr": 1e-3, "n_epochs": 500})
X_A = model(torch.tensor(X).to(device))[0].real

In [8]:
dlist = torch.cat([intrinsic_dimension(model, x) for x in DataLoader(X_A, batch_size=256)])
print(f"Average dimension found over dataset : {torch.mean(dlist, dtype=float).item()} (True: {d})")

Average dimension found over dataset : 30.0 (True: 10)


In [9]:
dlist = torch.cat([intrinsic_dimension(model, x, {"biggest_gap_diff": True}) for x in DataLoader(X_A, batch_size=256)])
print(f"Top-1 dimension found over dataset (with spectral gaps as differences): {torch.argmax(torch.bincount(dlist)).item()} (True: {d})")

Top-1 dimension found over dataset (with eigen gaps as differences): 9 (True: 10)


### MN1

18 dimensions non-linear manifold inside space of dimension 72.

In [ ]:
D = 72
d = 18
N = 16
n = 250
X = bm.generate(name='Mn1_Nonlinear', dim=D, d=d, noise=0.1, n=n)
model = QuantumCognitionModel(D, N, 0.).to(device)
model = train(model, X, {"lr": 1e-3, "n_epochs": 500})
X_A = model(torch.tensor(X).to(device))[0].real

In [13]:
dlist = torch.cat([intrinsic_dimension(model, x) for x in DataLoader(X_A, batch_size=256)])
print(f"Average dimension found over dataset : {torch.mean(dlist, dtype=float).item()} (True: {d})")

Average dimension found over dataset : 30.0 (True: 18)


In [14]:
dlist = torch.cat([intrinsic_dimension(model, x, {"biggest_gap_diff": True}) for x in DataLoader(X_A, batch_size=256)])
b = torch.bincount(dlist)
dim_top_1 = torch.argmax(b).item()
b[dim_top_1] = -1
dim_top_2 = torch.argmax(b).item()
print(f"Top-2 dimension found over dataset (with spectral gaps as differences): {dim_top_2} (True: {d})")

Top-2 dimension found over dataset (with spectral gaps as differences): 17 (True: 18)


## Observations :

- On the two first experiments the agreement between the dimension found and expected is good (1.9x for 2; 16.9x for 17).

- On the last two experiments, if we take stricly the method of the paper, we don't find their results. However, when tweaking slightly the method we find dimensions very close to what was expected (9 -> 10; and 17 -> 18). This is very promising.

Overall promising results. Potential next steps on our side to bridge the gap could be to robustify the training of the QCML model, probably with earlystopping on a validation set, and searching for the best learning rate.